# Analysis code for the stellar density fitting

This code provides a `Python` wrapper for the stellar density fitting performed using `Stan` for the paper "Stellar cores live long and prosper in cuspy dark matter halos". The fitting data used to make the figures in the paper is provided precomputed as well.

Alexander Rawlings (alexander.rawlings@helsinki.fi) 2025

In [ ]:
import os.path
import numpy as np
import matplotlib.pyplot as plt

# If cmdstanpy is not installed, uncomment and run these lines
# import sys
# !{sys.executable} -m pip install --user cmdstanpy
import cmdstanpy

In [ ]:
# run this if we need to install cmdstan, where the actual fitting is done
cmdstanpy.install_cmdstan(verbose=True)

In [ ]:
# change these variables to run different models, systems, etc. 
# file for single fit
run = 0
density_file = f"../data/density_profiles/fiducial_{run}/stellar_density_fiducial_{run}_000Gyr_cut95_nbins50.txt"

# list for hierarchical fit (uncomment and set hierarchical_model = True)
# density_file = [f"../data/density_profiles/fiducial_{run}/stellar_density_fiducial_{run}_000Gyr_cut95_nbins50.txt" for run in np.arange(10)]

output_file_name = "model_parameters.npz"
hierarchical_model = False

In [ ]:
assert os.path.splitext(output_file_name)[-1] == ".npz"

# these are the sampling parameters
sample_kwargs = {"adapt_delta": 0.995, "max_treedepth": 15}

# set the data the fit is done on
stan_data = {"N_obs":0, "N_group":0, "group_id":[], "r":[], "density":[]}
if hierarchical_model:
    assert isinstance(density_file, list)
    stan_file = "abg_hierarchy.stan"
    for f in density_file:
        data = np.loadtxt(f, skiprows=1)
        stan_data["r"].extend(data[:, 0])
        stan_data["density"].extend(data[:, 1])
        stan_data["N_obs"] += len(data)
        stan_data["N_group"] += 1
        stan_data["group_id"].extend(np.full(len(data), stan_data["N_group"][-1]))
else:
    assert isinstance(density_file, str)
    stan_file = "abg_simple.stan"
    data = np.loadtxt(density_file, skiprows=1)
    stan_data["r"].extend(data[:, 0])
    stan_data["density"].extend(data[:, 1])
    stan_data["N_obs"] += len(data)
    stan_data.pop("N_group")
    stan_data.pop("group_id")

# set the out-of-sample quantities
stan_data["N_OOS"] = 10 * stan_data["N_obs"]
rs = np.geomspace(1e-2, np.max(stan_data["r"]), 10 * len(data))
stan_data["r_OOS"] = rs
if hierarchical_model:
    ngroups = 2 * stan_data["N_group"]
    stan_data["N_group_OOS"] = ngroups
    stan_data["N_OOS"] *= ngroups
    stan_data["r_OOS"] = np.tile(rs, ngroups)
    stan_data["group_id_OOS"] = np.repeat(np.arange(1, ngroups + 1), len(rs))

# compile the model
model = cmdstanpy.CmdStanModel(
    stan_file=os.path.join(os.path.abspath(""), stan_file)
)

In [ ]:
# run the model
sample = model.sample(
    data = stan_data,
    **sample_kwargs
)

In [ ]:
# save the output
pars = {}
for p in ["rS_posterior", "a_posterior", "b_posterior", "g_posterior","log10rhoS_posterior"]:
    try:
        pars[p] = sample.stan_variable(p)
    except (KeyError, ValueError):
        pars[p] = sample.stan_variable(p.replace("_posterior", ""))
np.savez(output_file_name, **pars)
print(f"Saved OOS data to {output_file_name}")

In [ ]:
# check data was loaded
data = np.load(output_file_name)
for k in data:
    print(k, data[k].shape)